# Coordinate Descent

### Lower Bound, Take 4

Ensure feasibility of "Lower Bound, Take 2" by adjusting alpha as necessary.

### Lower Bound, Take 3

Ensure feasibility by allowing a stochastic mixture with the MLE.  Doesn't work (not DCP).

Assume $r_{\min} = 0$ for simplicity.  Idea for online solving $$
\begin{aligned}
&\!\min_{Q \succeq 0} &\qquad& \sum_{(w,r)} w r Q_{w, r}, \\
&\text{subject to} &  & \sum_{(w,r)} w Q_{w,r} = 1, \\
&                  &  & \sum_{(w,r)} Q_{w,r} = 1, \\
&                  &  & \sum_n \log(Q_{w_n, r_n}) \geq \phi
\end{aligned}
$$ where $\phi = -\frac{1}{2} \chi^{2,\alpha}_{(1)} + \sum_n \log(Q^{\text{(mle)}}_{w_n, r_n})$.  Because the support of $Q$ is at most the empirical support plus $(w_{\min}, 0)$ and $(w_{\max}, 0)$ we will maintain two variables $q_{\min}$ and $q_{\max}$ corresponding to $w_{\min}$ and $w_{\max}$ respectively.  Otherwise we need one primal variable for each data point.  However we will use two primal variables corresponding to $r = 0$ and $r = 1$ respectively.  We will split each datapoint into two points whose fractional counts are proportional to $r$ and $(1 - r)$ respectively.

At time $t$ we receive $(w_t, r_t)$ and we want to determine $q_t$.  We are allowed to choose the new $q_{\min}$ and $q_{\max}$ arbitrarily.  For points $q_{<t}$ we are allowed to scale them by $\psi_0$ and $\psi_1$, corresponding to points with $r = 0$ and $r = 1$ respectively.  We are also allowed to stochastically mix in the maximum likelihood solution according to $\psi_{\text{mle}}$.  We assume $(q_{0,<t}, q_{1,<t}, q_{<t,\min}, q_{<t,\max})$ is feasible before receiving $(w_t, r_t)$.  Then 

$$
\begin{aligned}
&\!\min_{q_{0,t}, q_{1,t}, q_{\min}, q_{\max}, \psi_0, \psi_1 \succeq 0, \psi_{\text{mle}} \in [0, 1]} &\qquad& q_{1,t} w_t + \psi_1 v_{<t} + \psi_{\text{mle}} v_{\text{mle}} \\
&\text{subject to} &  & w_t q_{0,t} + w_t q_{1,t} + w_{\min} q_{\min} + w_{\max} q_{\max} + \psi_0 w_{0,<t} q_{0,<t} + \psi_1 w_{1,<t} q_{1,<t} = 1, \\
&                  &  & q_{0,t} + q_{1,t} + q_{\min} + q_{\max} + \psi_0 q_{0,<t} + \psi_1 q_{1,<t} = 1, \\
&                  &  & (1 - r_t) \log(q_{0,t}) + r_t \log(q_{1,t}) + (t - r_{<t}) \log(\psi_0) + r_{<t} \log(\psi_1) \geq \phi - \mathcal{L}_{<t} = -\frac{1}{2} \chi^{2,\alpha}_{(1)} + \mathcal{L}^{\text{(mle)}}_t - \mathcal{L}_{<t}
\end{aligned}
$$ where $v_{<t}$ is the previous lower bound, $\mathcal{L}^{\text{(mle)}}_t$ is the mle likelihood of the observed data including point $t$, and $\mathcal{L}_{<t}$ is the previously obtained likelihood of the observed data for the lower bound.

### Lower Bound, Take 2

Better, but still has feasibility issues as the stream progresses.

Assume $r_{\min} = 0$ for simplicity.  Idea for online solving $$
\begin{aligned}
&\!\min_{Q \succeq 0} &\qquad& \sum_{(w,r)} w r Q_{w, r}, \\
&\text{subject to} &  & \sum_{(w,r)} w Q_{w,r} = 1, \\
&                  &  & \sum_{(w,r)} Q_{w,r} = 1, \\
&                  &  & \sum_n \log(Q_{w_n, r_n}) \geq \phi
\end{aligned}
$$ where $\phi = -\frac{1}{2} \chi^{2,\alpha}_{(1)} + \sum_n \log(Q^{\text{(mle)}}_{w_n, r_n})$.  Because the support of $Q$ is at most the empirical support plus $(w_{\min}, 0)$ and $(w_{\max}, 0)$ we will maintain two variables $q_{\min}$ and $q_{\max}$ corresponding to $w_{\min}$ and $w_{\max}$ respectively.  Otherwise we need one primal variable for each data point.  

At time $t$ we receive $(w_t, r_t)$ and we want to determine $q_t$.  We are allowed to choose the new $q_{\min}$ and $q_{\max}$ arbitrarily.  For points $q_{<t}$ we are allowed to scale them by $\psi_0$ and $\psi_1$, corresponding to points with $r = 0$ and $r = 1$ respectively.  We assume $(q_{0,<t}, q_{1,<t}, q_{<t,\min}, q_{<t,\max})$ is feasible before receiving $(w_t, r_t)$.  Then 

$$
\begin{aligned}
&\!\min_{q_t, q_{\min}, q_{\max},\psi_0, \psi_1 \succeq 0} &\qquad& q_t w_t r_t + \psi_1 v_{<t} \\
&\text{subject to} &  & w_t q_t + w_{\min} q_{\min} + w_{\max} q_{\max} + \psi_0 w_{0,<t} q_{0,<t} + \psi_1 w_{1,<t} q_{1,<t} = 1, \\
&                  &  & q_t + q_{\min} + q_{\max} + \psi_0 q_{0,<t} + \psi_1 q_{1,<t} = 1, \\
&                  &  & \log(q_t) + (t - r_{<t}) \log(\psi_0) + r_{<t} \log(\psi_1) \geq \phi - \mathcal{L}_{<t} = -\frac{1}{2} \chi^{2,\alpha}_{(1)} + \mathcal{L}^{\text{(mle)}}_t - \mathcal{L}_{<t}
\end{aligned}
$$ where $v_{<t}$ is the previous lower bound, $\mathcal{L}^{\text{(mle)}}_t$ is the mle likelihood of the observed data including point $t$, and $\mathcal{L}_{<t}$ is the previously obtained likelihood of the observed data for the lower bound.

After computing $q_t$ we increment $q_{0,<t}$ and $q_{1,<t}$ proportional to $1 - r_t$ and $r_t$ respectively; and $w_{0,<t}$ and $w_{1,<t}$ by $w_t (1 - r_t)$ and $w_t r_t$ respectively.

### Lower Bound

Starts out good, but then runs into infeasibility.

Assume $r_{\min} = 0$ for simplicity.  Idea for online solving $$
\begin{aligned}
&\!\min_{Q \succeq 0} &\qquad& \sum_{(w,r)} w r Q_{w, r}, \\
&\text{subject to} &  & \sum_{(w,r)} w Q_{w,r} = 1, \\
&                  &  & \sum_{(w,r)} Q_{w,r} = 1, \\
&                  &  & \sum_n \log(Q_{w_n, r_n}) \geq \phi
\end{aligned}
$$ where $\phi = -\frac{1}{2} \chi^{2,\alpha}_{(1)} + \sum_n \log(Q^{\text{(mle)}}_{w_n, r_n})$.  Because the support of $Q$ is at most the empirical support plus $(w_{\min}, 0)$ and $(w_{\max}, 0)$ we will maintain two variables $q_{\min}$ and $q_{\max}$ corresponding to $w_{\min}$ and $w_{\max}$ respectively.  Otherwise we need one primal variable for each data point.

At time $t$ we receive $(w_t, r_t)$ and we want to determine $q_t$.  We are allowed to choose the new $q_{\min}$ and $q_{\max}$ arbitrarily.  For points $q_{<t}$ we only allowed to scale them by $\psi$.  We assume $(q_{<t}, q_{<t,\min}, q_{<t,\max})$ is feasible before receiving $(w_t, r_t)$.  Then 

$$
\begin{aligned}
&\!\min_{q_t, q_{\min}, q_{\max}, \psi \succeq 0} &\qquad& q_t w_t r_t + \psi v_{<t} \\
&\text{subject to} &  & w_t q_t + w_{\min} q_{\min} + w_{\max} q_{\max} + \psi w_{<t} q_{<t} = 1, \\
&                  &  & q_t + q_{\min} + q_{\max} + \psi q_{<t} = 1, \\
&                  &  & \log(q_t) + t \log(\psi) \geq \phi - \mathcal{L}_{<t} = -\frac{1}{2} \chi^{2,\alpha}_{(1)} + \mathcal{L}^{\text{(mle)}}_t - \mathcal{L}_{<t}
\end{aligned}
$$ where $v_{<t}$ is the previous lower bound, $\mathcal{L}^{\text{(mle)}}_t$ is the mle likelihood of the observed data including point $t$, and $\mathcal{L}_{<t}$ is the previously obtained likelihood of the observed data for the lower bound. Substituting $q_{<t} + q_{<t,\min} + q_{<t,\max} = 1$ and $w_{<t} q_{<t} + w_{\min} q_{<t,\min} + w_{\max} q_{<t,\max} = 1$ yields 

$$
\begin{aligned}
&\!\min_{q_t, q_{\min}, q_{\max} \succeq 0, \psi \in [0, 1]} &\qquad& q_t w_t r_t + \psi v_{<t} \\
&\text{subject to} &  & w_t q_t + w_{\min} q_{\min} + w_{\max} q_{\max} + \psi (1 - w_{\min} q_{<t,\min} - w_{\max} q_{<t,\max}) = 1, \\
&                  &  & q_t + q_{\min} + q_{\max} + \psi (1 - q_{<t,\max} - q_{<t,\min}) = 1 \\
&                  &  & -\log(q_t) - t \log(\psi) - \frac{1}{2} \chi^{2,\alpha}_{(1)} + \mathcal{L}^{\text{(mle)}}_t  - \mathcal{L}_{<t} \leq 0
\end{aligned}
$$


### MLE

Idea for online solving $$
\begin{aligned}
&\!\max_{Q \succeq 0} &\qquad& \sum_n \log(Q_{w_n, r_n}), \\
&\text{subject to} &  & \sum_{(w,r)} w Q_{w,r} = 1, \\
&                  &  & \sum_{(w,r)} Q_{w,r} = 1. 
\end{aligned}
$$
Because the support of $Q$ is at most the empirical support plus $w_{\min}$ and $w_{\max}$ we will maintain two variables $q_{\min}$ and $q_{\max}$ corresponding to $w_{\min}$ and $w_{\max}$ respectively.  Otherwise we need one primal variable for each data point.

At time $t$ we receive $(w_t, r_t)$ and we want to determine $q_t$.  We are allowed to choose the new $q_{\min}$ and $q_{\max}$ arbitrarily.  For points $q_{<t}$ we only allowed to scale them by $\psi$.  We assume $(q_{<t}, q_{<t,\min}, q_{<t,\max})$ is feasible before receiving $(w_t, r_t)$.  Then 

$$
\begin{aligned}
&\!\max_{q_t, q_{\min}, q_{\max}, \psi > 0} &\qquad& t \log(\psi) + \log(q_t), \\
&\text{subject to} &  & w_t q_t + w_{\min} q_{\min} + w_{\max} q_{\max} + \psi w_{<t} q_{<t} = 1, \\
&                  &  & q_t + q_{\min} + q_{\max} + \psi q_{<t} = 1
\end{aligned}
$$ 

Substituting $q_{<t} + q_{<t,\min} + q_{<t,\max} = 1$ and $w_{<t} q_{<t} + w_{\min} q_{<t,\min} + w_{\max} q_{<t,\max} = 1$ yields 

$$
\begin{aligned}
&\!\max_{q_t, q_{\min}, q_{\max}, \psi > 0} &\qquad& t \log(\psi) + \log(q_t), \\
&\text{subject to} &  & w_t q_t + w_{\min} q_{\min} + w_{\max} q_{\max} + \psi (1 - w_{\min} q_{<t,\min} - w_{\max} q_{<t,\max}) = 1, \\
&                  &  & q_t + q_{\min} + q_{\max} + \psi (1 - q_{<t,\max} - q_{<t,\min}) = 1 
\end{aligned}
$$

At the beginning of time we can initialize with $$
\begin{aligned}
q_{0,\min} &= \frac{1 - w_{\min}}{w_{\max} - w_{\min}} \\
q_{0,\max} &= \frac{w_{\max} - 1}{w_{\max} - w_{\min}}
\end{aligned}
$$

### Code

In [344]:
class OnlineCoordinateDescentMLE:
    def __init__(self, wmin, wmax):
        from cvxopt import matrix

        assert wmax > 1
        assert wmin >= 0
        assert wmin < wmax
        
        self.wmin = wmin
        self.wmax = wmax
        self.qmin = (wmax - 1) / (wmax - wmin)
        self.qmax = (1 - wmin) / (wmax - wmin)
        self.obj = 0
        self.vmin = 0
        self.lastphi = 0
        
        self.G = matrix([ [ -1, 0, 0, 0 ],
                          [ 0, -1, 0, 0 ],
                          [ 0, 0, -1, 0 ],
                          [ 0, 0, 0, -1 ],
                        ],
                        tc='d').T
        self.h = matrix([ 0, 0, 0, 0 ], tc='d')
        self.b = matrix([ 1 / wmax, 1 ], tc='d')        
        self.t = 0
        
    def update(self, c, w, r):
        from cvxopt import matrix, solvers
        
        assert c > 0
    
        safet = max(self.t, 1)
        x0 = matrix([ c / (c + safet), 
                      self.qmin * safet / (c + safet), 
                      self.qmax * safet / (c + safet), 
                      safet / (c + safet) ], 
                    tc='d')

        def F(x=None, z=None):
            import math
            
            if x is None: return 0, x0
            
            if x[0] <= 0 or x[3] <= 0:
                return None
            
            f = -c * math.log(x[0]) / safet - self.t * math.log(x[3]) / safet
            jf = matrix([ -c / (safet * x[0]), 0, 0, -self.t / (safet * x[3]) ], tc='d').T
            if z is None: return f, jf
            hf = z[0] * matrix([ [ (c / safet) * 1/x[0]**2, 0, 0, 0 ], 
                                 [ 0, 0, 0, 0 ],
                                 [ 0, 0, 0, 0 ],
                                 [ 0, 0, 0, (self.t / safet) * 1/x[3]**2 ] 
                               ], tc='d')
            return f, jf, hf
          
        A = matrix([ 
                     [ float(w) / self.wmax, 
                       self.wmin / self.wmax, 
                      1, 
                      (1 / self.wmax - (self.wmin / self.wmax) * self.qmin - self.qmax) ],
                     [ 1, 1, 1, (1 - self.qmin - self.qmax) ]
                   ],
                   tc='d')
                
        soln = solvers.cp(F=F, G=self.G, h=self.h, A=A.T, b=self.b, options={'show_progress': False})
        from pprint import pformat
        assert soln['status'] == 'optimal', pformat([ soln, self.t ])
        
        self.obj -= safet * soln['primal objective']
        self.lastq = soln['x'][0]
        self.qmin = soln['x'][1]
        self.qmax = soln['x'][2]
        self.lastphi = soln['x'][3]
        self.vmin = soln['x'][0] * w * r + soln['x'][3] * self.vmin
        self.t += c
        
        return self.lastq

class OnlineCoordinateDescentLB: 
    class Flass:
        def __init__(self):
            pass
        
    def __init__(self, wmin, wmax, alpha): 
        assert wmax > 1
        assert wmin >= 0
        assert wmin < wmax
        
        self.wmin = wmin
        self.wmax = wmax
        self.qmin = (wmax - 1) / (wmax - wmin)
        self.qmax = (1 - wmin) / (wmax - wmin)
        
        self.vlb = 0
        self.wq0 = 0
        self.wq1 = 0
        self.q0t = 0
        self.q1t = 0
        self.t = 0
        self.rt = 0
        self.llb = 0
        
        self.alpha = alpha
        self.mle = OnlineCoordinateDescentMLE(wmin=wmin, wmax=wmax)
        
        from scipy.stats import chi2
        import cvxpy as cp
        
        self.vars = OnlineCoordinateDescentLB.Flass()
        self.vars.qt = cp.Variable(nonneg=True)
        self.vars.qmin = cp.Variable(nonneg=True)
        self.vars.qmax = cp.Variable(nonneg=True)
        self.vars.psi0 = cp.Variable(nonneg=True)
        self.vars.psi1 = cp.Variable(nonneg=True)
        
        self.params = OnlineCoordinateDescentLB.Flass()
        self.params.w = cp.Parameter(nonneg=True)
        self.params.wr = cp.Parameter(nonneg=True)
        self.params.vlb = cp.Parameter(nonneg=True)
        self.params.wq0 = cp.Parameter(nonneg=True)
        self.params.wq1 = cp.Parameter(nonneg=True)
        self.params.q0t = cp.Parameter(nonneg=True)
        self.params.q1t = cp.Parameter(nonneg=True)
        self.params.c = cp.Parameter(nonneg=True)
        self.params.tminusrt = cp.Parameter(nonneg=True)
        self.params.rt = cp.Parameter(nonneg=True)
        self.params.constraintrhs = cp.Parameter()
        
        self.prob = cp.Problem(cp.Minimize(self.params.wr * self.vars.qt + self.params.vlb * self.vars.psi1), [
              self.params.w * self.vars.qt
            + (self.wmin / self.wmax) * self.vars.qmin 
            + self.vars.qmax 
            + self.params.wq0 * self.vars.psi0 
            + self.params.wq1 * self.vars.psi1
            == 1 / self.wmax,  
              self.vars.qt 
            + self.vars.qmin 
            + self.vars.qmax 
            + self.params.q0t * self.vars.psi0
            + self.params.q1t * self.vars.psi1 == 1,
              self.params.c * cp.log(self.vars.qt) 
            + self.params.tminusrt * cp.log(self.vars.psi0)
            + self.params.rt * cp.log(self.vars.psi1) 
            >= self.params.constraintrhs
        ])
        
    def innersolve(self, c, w, r, alpha):
        from scipy.stats import chi2

        safet = max(self.t, 1)
        halfchisq = 0.5 * chi2.isf(q=alpha, df=1)
        
        self.params.w.value = w / self.wmax
        self.params.wr.value = w * r
        self.params.vlb.value = self.vlb
        self.params.wq0.value = self.wq0 / self.wmax
        self.params.wq1.value = self.wq1 / self.wmax
        self.params.q0t.value = self.q0t
        self.params.q1t.value = self.q1t
        self.params.c.value = c / safet
        self.params.tminusrt.value = (self.t - self.rt) / safet
        self.params.rt.value = self.rt / safet
        self.params.constraintrhs.value = (-halfchisq + self.mle.obj - self.llb) / safet
        
        self.prob.solve(verbose=False)
        
        return (self.prob.value, 
                self.prob.status, 
                (1 - r) * self.vars.qt.value if self.vars.qt.value is not None else None, 
                r * self.vars.qt.value if self.vars.qt.value is not None else None, 
                self.vars.qmin.value, 
                self.vars.qmax.value, 
                self.vars.psi0.value, 
                self.vars.psi1.value
               )
        
    def updatev3(self, c, w, r):
        import cvxpy as cp
        import math
        from scipy.special import xlogy
        
        qmle = self.mle.update(c, w, r)
        
        q0t = cp.Variable(nonneg=True)
        q1t = cp.Variable(nonneg=True)
        qmin = cp.Variable(nonneg=True)
        qmax = cp.Variable(nonneg=True)
        psi0 = cp.Variable(nonneg=True)
        psi1 = cp.Variable(nonneg=True)
        psimle = cp.Variable(nonneg=True)
        
        safet = max(self.t, 1)

#        prob = cp.Problem(cp.Minimize((float(w) * q1t + self.vlb * psi1) * (1 - psimle) + self.mle.vhat * psimle), [
        prob = cp.Problem(cp.Minimize(float(w) * q1t + self.vlb * psi1 + self.mle.vhat * psimle), [
              float(w / self.wmax) * q0t
            + float(w / self.wmax) * q1t
            + (self.wmin / self.wmax) * qmin 
            + qmax 
            + psi0 * (self.wq0 / self.wmax)
            + psi1 * (self.wq1 / self.wmax) 
            == 1 / self.wmax,  
            q0t + q1t + qmin + qmax + psi0 * self.q0t + psi1 * self.q1t == 1,
              float(c * (1 - r) / safet) * cp.log(q0t) 
            + float(c * r / safet) * cp.log(q1t) 
            + float((self.t - self.rt) / safet) * cp.log(psi0)
            + float(self.rt / safet) * cp.log(psi1) 
#             >= ((-self.halfchisq + self.mle.obj) / safet) * cp.inv_pos(1 - psimle) - (self.llb / safet),
            >= ((-self.halfchisq + self.mle.obj) / safet) * (1 + psimle) - (self.llb / safet),
            psimle <= 1,
            psimle >= 0.9
        ])
        prob.solve(verbose=False)
        assert prob.status[:7] == 'optimal', prob.solve(verbose=True)
        
        self.vlb = (w * q1t.value + self.vlb * psi1.value) * (1 - psimle.value) + psimle.value * self.mle.vhat
        self.qmin = qmin.value
        self.qmax = qmax.value
        self.llb += (  xlogy(c * (1 - r), q0t.value)
                     + xlogy(c * r, q1t.value)
                     + xlogy(self.t - self.rt, psi0.value)
                     + xlogy(self.rt, psi1.value)
                    )
        self.llb = (1 - psimle.value) * self.llb + psimle.value * self.mle.obj
     
        self.t += c
        self.rt += c * r
        self.q0t = q0t.value + psi0.value * self.q0t
        self.wq0 = w * q0t.value + psi0.value * self.wq0
        self.q1t = q1t.value + psi1.value * self.q1t
        self.wq1 = w * q1t.value + psi1.value * self.wq1
        
        myq = (1 - r) * q0t.value + r * q1t.value
        myq = (1 - psimle.value) * myq + psimle.value * qmle
        
        return myq, { 'self.q0t': self.q0t, 
                      'self.q1t': self.q1t,
                      'qmin': qmin.value,
                      'qmax': qmax.value,
                      'vlb': self.vlb,
                      'llb': self.llb,
                      'vmle': self.mle.vhat,
                      'self.rt': self.rt,
                      'psimle': psimle.value,
                    }

    def __initturg__(self, wmin, wmax, alpha): 
        assert wmax > 1
        assert wmin >= 0
        assert wmin < wmax
        
        self.wmin = wmin
        self.wmax = wmax
        self.qmin = (wmax - 1) / (wmax - wmin)
        self.qmax = (1 - wmin) / (wmax - wmin)
        
        self.vlb = 0
        self.wq0 = 0
        self.wq1 = 0
        self.q0t = 0
        self.q1t = 0
        self.t = 0
        self.rt = 0
        self.llb = 0
        
        self.alpha = alpha
        self.mle = OnlineCoordinateDescentMLE(wmin=wmin, wmax=wmax)
        
        from scipy.stats import chi2
        import cvxpy as cp
        
        self.vars = OnlineCoordinateDescentLB.Flass()
        self.vars.q0t = cp.Variable(nonneg=True)
        self.vars.q1t = cp.Variable(nonneg=True)
        self.vars.qmin = cp.Variable(nonneg=True)
        self.vars.qmax = cp.Variable(nonneg=True)
        self.vars.psi0 = cp.Variable(nonneg=True)
        self.vars.psi1 = cp.Variable(nonneg=True)
        
        self.params = OnlineCoordinateDescentLB.Flass()
        self.params.w = cp.Parameter(nonneg=True)
        self.params.wcost = cp.Parameter(nonneg=True)
        self.params.vlb = cp.Parameter(nonneg=True)
        self.params.wq0 = cp.Parameter(nonneg=True)
        self.params.wq1 = cp.Parameter(nonneg=True)
        self.params.q0t = cp.Parameter(nonneg=True)
        self.params.q1t = cp.Parameter(nonneg=True)
        self.params.coneminusr = cp.Parameter(nonneg=True)
        self.params.cr = cp.Parameter(nonneg=True)
        self.params.tminusrt = cp.Parameter(nonneg=True)
        self.params.rt = cp.Parameter(nonneg=True)
        self.params.constraintrhs = cp.Parameter()
        
        self.prob = cp.Problem(cp.Minimize(self.params.wcost * self.vars.q1t + self.params.vlb * self.vars.psi1), [
              self.params.w * self.vars.q0t
            + self.params.w * self.vars.q1t
            + (self.wmin / self.wmax) * self.vars.qmin 
            + self.vars.qmax 
            + self.params.wq0 * self.vars.psi0 
            + self.params.wq1 * self.vars.psi1
            == 1 / self.wmax,  
              self.vars.q0t 
            + self.vars.q1t 
            + self.vars.qmin 
            + self.vars.qmax 
            + self.params.q0t * self.vars.psi0
            + self.params.q1t * self.vars.psi1 == 1,
              self.params.coneminusr * cp.log(self.vars.q0t) 
            + self.params.cr * cp.log(self.vars.q1t) 
            + self.params.tminusrt * cp.log(self.vars.psi0)
            + self.params.rt * cp.log(self.vars.psi1) 
            >= self.params.constraintrhs
        ])

    def innersolveflass(self, c, w, r, alpha):   
        # doesn't work, not sure why (?)

        from cvxopt import matrix, spdiag, solvers
        import numpy as np
        from scipy.stats import chi2
        
        assert 0 < c
        assert 0 <= r
        assert r <= 1

        safet = max(self.t, 1)
        halfchisq = 0.5 * chi2.isf(q=alpha, df=1)
         
        print([
            w / self.wmax,
            w,
            self.vlb,
            self.wq0 / self.wmax,
            self.wq1 / self.wmax,
            self.q0t,
            self.q1t,
            c * (1 - r) / safet,
            c * r / safet,
            (self.t - self.rt) / safet,
            self.rt / safet,
            (-halfchisq + self.mle.obj - self.llb) / safet
        ])
        
        
        G = matrix(-np.eye(6), tc='d')
        h = matrix(0, size=(6,1), tc='d')
        
        A = matrix([ [ w / self.wmax,
                       w / self.wmax,
                       self.wmin / self.wmax, 
                       1, 
                       self.wq0 / self.wmax,
                       self.wq1 / self.wmax 
                     ],
                     [ 1, 1, 1, 1, self.q0t, self.q1t ]
                   ],
                   tc='d')
        b = matrix([ 1 / self.wmax, 1 ], tc='d')
        
        cost = matrix(0, size=(6,1), tc='d')
        cost[1] = float(w) 
        cost[5] = self.vlb 
        
        x0 = matrix([ c * (1 - r) / (c + safet),
                      c * r / (c + safet),
                      self.qmin * safet / (c + safet), 
                      self.qmax * safet / (c + safet), 
                      safet / (c + safet),
                      safet / (c + safet)
                    ], tc='d')

        def F(x=None, z=None):
            from scipy.special import xlogy

            if x is None: return 1, x0
            
            if any(z < 0 for z in x):
                 return None
                
            f = (  xlogy(c * (1 - r), x[0])
                 + xlogy(c * r, x[1])
                 + xlogy(self.t - self.rt, x[3])
                 + xlogy(self.rt, x[4])
                 + halfchisq
                 - self.mle.obj
                 + self.llb)
            f *= -1 / safet
            
            jf = matrix(0, size=(1, 6), tc='d')
            jf[0] = c * (1 - r) / x[0] if c * (1 - r) > 0 else 0 
            jf[1] = c * r / x[1] if c * r > 0 else 0 
            jf[4] = (self.t - self.rt) / x[4] if self.t > self.rt else 0
            jf[5] = self.rt / x[5] if self.rt > 0 else 0
            jf *= -1 / safet
            
            if z is None: return f, jf
            
            hf = spdiag([
                -c * (1 - r) / x[0]**2 if c * (1 - r) > 0 else 0,
                 -c * r / x[1]**2 if c * r > 0 else 0,               
                0,
                0,
                -(self.t - self.rt) / x[4]**2 if self.t > self.rt else 0,
                -self.rt / x[5]**2 if self.rt > 0 else 0 
            ]) 

            hf *= -z[0] / safet

            return f, jf, hf

        soln = solvers.cpl(c=cost, F=F, G=G, h=h, A=A.T, b=b)

        from pprint import pformat
        import numpy
        assert soln['status'][:7] == 'optimal', pformat({ 'soln': soln, 
                                                  'solnx': [ z for z in soln['x'] ], 
                                                  'datum': (c, w, r),
                                                  'F(x=x0)': F(x=x0),
                                                  'A': numpy.matrix(A),
                                                  'b': [ z for z in b ],
                                                  'A.x0 - b': [ z for z in A.T*x0 - b ],
                                                  'G.x0 - h': [ z for z in G*x0 - h ],
                                                  'F(x=soln)': F(x=soln['x']),
                                                  'A.x - b': [ z for z in A.T*soln['x'] - b ],
                                                  'G.x - h': [ z for z in G*soln['x'] - h ],
                                                })
        return (soln['primal objective'], soln['status'], 
                (1 - r) * soln['x'][0],
                r * soln['x'][0]
               ) + tuple(soln['x'][1:])

    def innersolveturg(self, c, w, r, alpha):
        from scipy.stats import chi2

        safet = max(self.t, 1)
        halfchisq = 0.5 * chi2.isf(q=alpha, df=1)
        
        self.params.w.value = w / self.wmax
        self.params.wcost.value = w
        self.params.vlb.value = self.vlb
        self.params.wq0.value = self.wq0 / self.wmax
        self.params.wq1.value = self.wq1 / self.wmax
        self.params.q0t.value = self.q0t
        self.params.q1t.value = self.q1t
        self.params.coneminusr.value = c * (1 - r) / safet
        self.params.cr.value = c * r / safet
        self.params.tminusrt.value = (self.t - self.rt) / safet
        self.params.rt.value = self.rt / safet
        self.params.constraintrhs.value = (-halfchisq + self.mle.obj - self.llb) / safet
        
        self.prob.solve(verbose=False)
        
        return (self.prob.value, 
                self.prob.status, 
                self.vars.q0t.value, 
                self.vars.q1t.value, 
                self.vars.qmin.value, 
                self.vars.qmax.value, 
                self.vars.psi0.value, 
                self.vars.psi1.value
               )
                
    def update(self, c, w, r):
        import math
        from scipy.special import xlogy
        
        self.mle.update(c, w, r)
        
        alpha = self.alpha
        
        (pvalue, pstatus, q0t, q1t, qmin, qmax, psi0, psi1) = self.innersolve(c, w, r, alpha)
        
        if pstatus[:7] != 'optimal':
            alphalb = 0
            alphaub = alpha
            
            while alphaub - alphalb >= 1e-3:
                alphatest = 0.5 * (alphalb + alphaub)
                (pvalue, pstatus, q0t, q1t, qmin, qmax, psi0, psi1) = self.innersolve(c, w, r, alphatest)
            
                if pstatus[:7] == 'optimal':
                    alphalb = alphatest
                else:
                    alphaub = alphatest
                    
            alpha = alphalb
            (pvalue, pstatus, q0t, q1t, qmin, qmax, psi0, psi1) = self.innersolve(c, w, r, alpha)
            
        assert pstatus[:7] == 'optimal', { 'alpha': alpha, 'pstatus': pstatus }
        
        self.vlb = pvalue
        self.qmin = qmin
        self.qmax = qmax
        self.llb += (  xlogy(c * (1 - r), q0t)
                     + xlogy(c * r, q1t)
                     + xlogy(self.t - self.rt, psi0)
                     + xlogy(self.rt, psi1)
                    )
     
        self.t += c
        self.rt += c * r
        self.q0t = q0t + psi0 * self.q0t
        self.wq0 = w * q0t + psi0 * self.wq0
        self.q1t = q1t + psi1 * self.q1t
        self.wq1 = w * q1t + psi1 * self.wq1
        
        return (1 - r) * q0t + r * q1t, { # 'self.q0t': self.q0t, 
#                                           'self.q1t': self.q1t,
#                                           'qmin': qmin.value,
#                                           'qmax': qmax.value,
                                            'vlb': self.vlb,
                                            'vmle': self.mle.vmin,
#                                           'self.rt': self.rt,
                                            'alpha': alpha,
                                        }

    def __initv1__(self, wmin, wmax, alpha):
        from scipy.stats import chi2
        from cvxopt import matrix

        assert wmax > 1
        assert wmin >= 0
        assert wmin < wmax
        
        self.wmin = wmin
        self.wmax = wmax
        self.qmin = (wmax - 1) / (wmax - wmin)
        self.qmax = (1 - wmin) / (wmax - wmin)
        
        self.G = matrix([ [ -1, 0, 0, 0 ],
                          [ 0, -1, 0, 0 ],
                          [ 0, 0, -1, 0 ],
                          [ 0, 0, 0, -1 ],
                        ],
                        tc='d').T
        self.h = matrix([ 0, 0, 0, 0 ], tc='d')
        self.b = matrix([ 1 / wmax, 1 ], tc='d')        
        self.t = 0

        self.halfchisq = 0.5 * chi2.isf(q=alpha, df=1)
        self.llb = 0
        self.vlb = 0
        
        self.mle = OnlineCoordinateDescentMLE(wmin=wmin, wmax=wmax)
    
    def updatev1(self, c, w, r):
        import cvxpy as cp
        import math
        
        self.mle.update(c, w, r)
                
        qt = cp.Variable(nonneg=True)
        qmin = cp.Variable(nonneg=True)
        qmax = cp.Variable(nonneg=True)
        psi = cp.Variable(nonneg=True)
        
        safet = max(self.t, 1)

        prob = cp.Problem(cp.Minimize(float(w * r) * qt + self.vlb * psi), [
              float(w / self.wmax) * qt 
            + (self.wmin / self.wmax) * qmin 
            + qmax 
            + (1 / self.wmax - (self.wmin / self.wmax) * self.qmin - self.qmax) * psi == 1 / self.wmax,
            qt + qmin + qmax + (1 - self.qmin - self.qmax) * psi == 1,
            float(c / safet) * cp.log(qt) + (self.t / safet) * cp.log(psi) >= (-self.halfchisq + self.mle.obj - self.llb) / safet
        ])
        prob.solve(verbose=False)
        
        if prob.status[:7] != 'optimal':
            # just maximize likelihood to recover (?)
            pass
        
        from pprint import pformat
        assert prob.status[:7] == 'optimal', pformat({ 'datum': [ c, w, r ], 
                                                   'x': [ z.value for z in [ qt, qmin, qmax, psi ] ],
                                                   'prob': prob.status,
                                                   't': self.t, 
                                                   'lmle': self.mle.obj, 
                                                   'llb': self.llb,# + c * math.log(qt.value) + self.t * math.log(psi.value),
                                                   'halfchisq': self.halfchisq,
                                                 })
        
        self.vlb = prob.value
        self.qmin = qmin.value
        self.qmax = qmax.value
        self.lastphi = psi.value
        self.llb += c * math.log(qt.value) + self.t * math.log(psi.value)
        self.t += c

        return { 'soln': [ z.value for z in [ qt, qmin, qmax, psi ] ], 'lmle': self.mle.obj, 
                 'llb': self.llb, 'halfchisq': self.halfchisq, 'vlb': self.vlb, 'vhat': self.mle.vhat }
       
        
        if False:
            from cvxopt import matrix, solvers
            import math

            assert c > 0 

            self.mle.update(c, w, r)
            lmle = self.mle.obj

            safet = max(self.t, 1)
            x0 = matrix([ c / (c + safet), 
                          self.qmin * safet / (c + safet), 
                          self.qmax * safet / (c + safet), 
                          safet / (c + safet) ], tc='d')

            def F(x=None, z=None):
                if x is None: return 1, x0

                if x[0] <= 0 or x[3] <= 0:
                     return None

                f = -c * math.log(x[0]) - self.t * math.log(x[3]) - self.halfchisq + lmle - self.llb
                f /= safet
                jf = matrix([ -c / (safet * x[0]), 0, 0, -self.t / (safet * x[3]) ], tc='d').T
                if z is None: return f, jf
                hf = z[0] * matrix([ [ (c / safet) * 1/x[0]**2, 0, 0, 0 ], 
                                     [ 0, 0, 0, 0 ],
                                     [ 0, 0, 0, 0 ],
                                     [ 0, 0, 0, (self.t / safet) * 1/x[3]**2 ] 
                                   ], tc='d')
                return f, jf, hf

            cost = matrix([ float(w * r), 0, 0, self.vlb ], tc='d')
            A = matrix([ 
                         [ float(w) / self.wmax, 
                           self.wmin / self.wmax, 
                          1, 
                          (1 / self.wmax - (self.wmin / self.wmax) * self.qmin - self.qmax) ],
                         [ 1, 1, 1, (1 - self.qmin - self.qmax) ]
                       ],
                       tc='d')

            soln = solvers.cpl(c=cost, F=F, G=self.G, h=self.h, A=A.T, b=self.b, options={'show_progress': True, 
                                                                                          'maxiters': 100 })
            from pprint import pformat
            assert soln['status'] == 'optimal', pformat({ 'datum': [ c, w, r ], 
                                                          'x': [ z for z in soln['x'] ], 
                                                          'soln': soln,
                                                          't': self.t, 
                                                          'lmle': lmle, 
                                                          'llb': self.llb, 
                                                          'F(x0)': F(x=x0),
                                                          'F(x)': F(soln['x'])
                                                        })
            print(pformat((F(x=x0), F(x=soln['x']))))

            self.vlb = soln['primal objective']
            self.qmin = soln['x'][1]
            self.qmax = soln['x'][2]
            self.lastphi = soln['x'][3]
            self.llb += c * math.log(soln['x'][0]) + self.t * math.log(soln['x'][3])
            self.t += c

            return { 'soln': [ z for z in soln['x'] ], 'lmle': lmle, 
                     'llb': self.llb, 'halfchisq': self.halfchisq, 'vlb': self.vlb, 'vhat': self.mle.vhat }

class Test:
    def flass():
        import environments.ControlledRangeVariance
        import MLE.MLE
        
        def batchtoonline(samples, seed=45):
            import numpy as np

            state = np.random.RandomState(seed)

            n = sum(c for c, w, r in samples)
            while n > 0:
                p = np.array([ c for c, w, r in samples ], dtype='float64') / n
                what = state.choice(len(samples), p=p)
                c = min(samples[what][0], 1)
                yield (c, samples[what][1], samples[what][2])
                samples[what] = (samples[what][0] - c, samples[what][1], samples[what][2]) 
                n -= c
                
        env = environments.ControlledRangeVariance.ControlledRangeVariance(seed=45, wsupport=[0,2,1000], expwsq=100)
        _, samples = env.sample(1 + (1 << 16))
        ocd = OnlineCoordinateDescentLB(wmin=0, wmax=1000, alpha=0.05)

        from pprint import pformat
        
        t = 0
        for n, (c, w, r) in enumerate(batchtoonline(samples)):
            qt, qex = ocd.update(c, w, r)
            t += c
            if (n & (n - 1) == 0 and n & 0xAAAAAAAA == 0) or w == 1000:
                vlb = '{:.5}'.format(qex['vlb'])
                vmle = '{:.5}'.format(qex['vmle'])
                print([ c, w, r, t, '{:.3g}'.format(qt), '{:.3f}'.format(t * qt), { 'vlb': vlb, 'vmle': vmle, 'alpha': qex['alpha'] } ], 
                      flush=True)

Test.flass()

[1, 2, 1, 1, '0.0733', '0.073', {'vlb': '0.1465', 'vmle': '1.0', 'alpha': 0.05}]
[1, 2, 0, 2, '0.481', '0.962', {'vlb': '0.038075', 'vmle': '0.5', 'alpha': 0.05}]
[1, 0, 0, 5, '0.247', '1.236', {'vlb': '0.021139', 'vmle': '0.3', 'alpha': 0.05}]
[1, 2, 1, 17, '0.0223', '0.379', {'vlb': '0.22307', 'vmle': '0.5593', 'alpha': 0.05}]
[1, 2, 1, 65, '0.0102', '0.662', {'vlb': '0.42774', 'vmle': '0.63908', 'alpha': 0.05}]
[1, 2, 0, 257, '0.00427', '1.097', {'vlb': '0.6275', 'vmle': '0.74588', 'alpha': 0.05}]
[1, 2, 1, 1025, '0.000895', '0.917', {'vlb': '0.66016', 'vmle': '0.72023', 'alpha': 0.05}]
[1, 1000, 0, 3805, '0.000129', '0.489', {'vlb': '0.67922', 'vmle': '0.71246', 'alpha': 0.05}]
[1, 0, 0, 4097, '0.00025', '1.025', {'vlb': '0.67433', 'vmle': '0.70771', 'alpha': 0.05}]
[1, 2, 1, 16385, '5.97e-05', '0.978', {'vlb': '0.71508', 'vmle': '0.73192', 'alpha': 0.05}]
[1, 1000, 1, 23861, '5.33e-06', '0.127', {'vlb': '0.73212', 'vmle': '0.81397', 'alpha': 0.05}]
[1, 1000, 1, 30958, '7.18e-06', 

# Histogram Based

In [353]:
from importlib import reload
import environments.ControlledRangeVariance
import MLE.MLE 

reload(environments.ControlledRangeVariance)
reload(MLE.MLE)

def batchtoonline(samples, seed=45):
    import numpy as np
    
    state = np.random.RandomState(seed)
    
    n = sum(c for c, w, r in samples)
    while n > 0:
        p = np.array([ c for c, w, r in samples ], dtype='float64') / n
        what = state.choice(len(samples), p=p)
        c = min(samples[what][0], 1)
        yield (c, samples[what][1], samples[what][2])
        samples[what] = (samples[what][0] - c, samples[what][1], samples[what][2]) 
        n -= c
    
env = environments.ControlledRangeVariance.ControlledRangeVariance(seed=45, wsupport=[0,2,1000], expwsq=100)
happrox = MLE.MLE.Online.HistApprox(wmin=0, wmax=1000, numbuckets=10)
onlineci = MLE.MLE.Online.CI(wmin=0, wmax=1000, rmin=0, rmax=1, alpha=0.05)
onlinemle = MLE.MLE.Online.MLE(wmin=0, wmax=1000, rmin=0, rmax=1)
_, samples = env.sample(1 + (1 << 16))
from pprint import pformat
print(pformat(samples), flush=True)

t = 0
for n, (c, w, r) in enumerate(batchtoonline(samples)):
    happrox.update(c, w, r)
    onlineci.update(happrox.iterator)
    onlinemle.update(happrox.iterator)
    t += c
    
    if (n & (n - 1) == 0 and n & 0xAAAAAAAA == 0) or w == 1000:
        vmle = MLE.MLE.estimate(happrox.iterator, wmin=0, wmax=1000)[1]['vmin']
        vlb = MLE.MLE.asymptoticconfidenceinterval(happrox.iterator, wmin=0, wmax=1000)[0][0]
        print([ c, w, r, t, 'n/a', 'n/a', { 'vlb': '{:.3f}'.format(vlb), 'vmle': '{:.3f}'.format(vmle), 'alpha': 0.05 } ], 
                              flush=True)
        
#         from pprint import pformat
#         print(pformat(
#                 {
#                     'n': n,
#                     'onlineci': onlineci.getqfunc(),
#                     'onlinemle': onlinemle.getqfunc(),
#                     'batchmle': MLE.MLE.estimate(happrox.iterator, wmin=0, wmax=1000), 
#                     'batchci': MLE.MLE.asymptoticconfidenceinterval(happrox.iterator, wmin=0, wmax=1000)
#                 }), 
#               flush=True)

[(35819, 0, 0), (5658, 2, 0), (24056, 2, 1), (2, 1000, 0), (2, 1000, 1)]
[1, 2, 1, 1, 'n/a', 'n/a', {'vlb': '0.000', 'vmle': '1.000', 'alpha': 0.05}]
[1, 2, 0, 2, 'n/a', 'n/a', {'vlb': '-0.000', 'vmle': '0.500', 'alpha': 0.05}]
[1, 0, 0, 5, 'n/a', 'n/a', {'vlb': '0.003', 'vmle': '0.400', 'alpha': 0.05}]
[1, 2, 1, 17, 'n/a', 'n/a', {'vlb': '0.208', 'vmle': '0.589', 'alpha': 0.05}]
[1, 2, 1, 65, 'n/a', 'n/a', {'vlb': '0.425', 'vmle': '0.647', 'alpha': 0.05}]
[1, 2, 0, 257, 'n/a', 'n/a', {'vlb': '0.625', 'vmle': '0.748', 'alpha': 0.05}]
[1, 2, 1, 1025, 'n/a', 'n/a', {'vlb': '0.659', 'vmle': '0.721', 'alpha': 0.05}]
[1, 1000, 0, 3805, 'n/a', 'n/a', {'vlb': '0.678', 'vmle': '0.710', 'alpha': 0.05}]
[1, 0, 0, 4097, 'n/a', 'n/a', {'vlb': '0.675', 'vmle': '0.706', 'alpha': 0.05}]
[1, 2, 1, 16385, 'n/a', 'n/a', {'vlb': '0.716', 'vmle': '0.731', 'alpha': 0.05}]
[1, 1000, 1, 23861, 'n/a', 'n/a', {'vlb': '0.733', 'vmle': '0.782', 'alpha': 0.05}]
[1, 1000, 1, 30958, 'n/a', 'n/a', {'vlb': '0.750', '